In [102]:
from jira import JIRA
import csv
import pandas as pd
import numpy as np
import pandas_gbq
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from tqdm import tqdm
import time
import pygsheets
import datetime as dt
from datetime import datetime
import time
# Multi-processing
from multiprocessing import Pool, cpu_count
# Disable INFO logging
import logging
import warnings
warnings.filterwarnings("ignore")


username = 'lindani@loop.co.za'
api_token = 'ATATT3xFfGF0b4F8ivyIlhzX7qxYDrXZLbJcIjHAR2UD8qdfYi_NHjQBl-GoyJqF5bnYVBTKxnTSgj4SNoYoCh0urpLeZVSEg1hP26lRI55xj15nRueYNJEYZuWWrnwCVD49WIhA4cT5kx3aFsb89_if_UIGIsRIZwEkmZ5LqdFknLY4sql3AtI=8038D4C3'

# set up JIRA connection
jira_options = {'server': 'https://cowabunga-tasks.atlassian.net'}
jira = JIRA(options=jira_options, basic_auth=(username, api_token))

# JQL query to retrieve issues
jql_query = 'project = LOOP'


credentials = Credentials.from_service_account_file('C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json')



# fields to retrieve
fields = ['key', 'created', 'assignee', 'summary', 'reporter', 'issuetype', 'status', 'resolution', 
          'resolutiondate', 'updated','Task Type', 'priority', 'labels', 'Start date', 'Developer/s', 
          'Dev finished date', 'Went live / completed date', 'Development Effort', 'Environment/System', 
          'Flagged', 'Abandoned date', 'Abandoned Reason']

# retrieve issues and create DataFrame
issues = jira.search_issues(jql_query, fields=fields, maxResults=None)
issue_list = []
for issue in issues:
    developer_field = issue.fields.customfield_10053 if hasattr(issue.fields, 'customfield_10053') else None
    developer_value = ", ".join([developer.displayName for developer in developer_field]) if developer_field else None
    labels_field = issue.fields.labels if hasattr(issue.fields, 'labels') else None
    labels_value = ", ".join(labels_field) if labels_field else None
    issue_dict = {
        'Key': issue.key,
        'Assignee': issue.fields.assignee.displayName if issue.fields.assignee else None,
        'Created': issue.fields.created,
        'Reporter': issue.fields.reporter.displayName if issue.fields.reporter else None,
        'Status': issue.fields.status.name if issue.fields.status else None,
        'Resolution': issue.fields.resolution.name if issue.fields.resolution else None,
        'Resolved': issue.fields.resolutiondate,
        'Priority': issue.fields.priority.name if issue.fields.priority else None,
        'Labels': labels_value,
        'Updated': issue.fields.updated,
        'Summary': issue.fields.summary,
        'Issue Type': issue.fields.issuetype.name if issue.fields.issuetype else None,
        'Task Type': issue.fields.customfield_10051 if hasattr(issue.fields, 'customfield_10051') else None,
        'Start date': issue.fields.customfield_10015 if hasattr(issue.fields, 'customfield_10015') else None,
        'Developer/s': developer_value,
        'Dev finished date': issue.fields.customfield_10048 if hasattr(issue.fields, 'customfield_10048') else None,
        'Went live / completed date': issue.fields.customfield_10054 if hasattr(issue.fields, 'customfield_10054') else None,
        'Development Effort': issue.fields.customfield_10055 if hasattr(issue.fields, 'customfield_10055') else None,
        'Environment/System': issue.fields.customfield_10052 if hasattr(issue.fields, 'customfield_10052') else None,
        'Flagged': issue.fields.customfield_10021 if hasattr(issue.fields, 'customfield_10021') else None,
        'Abandoned date': issue.fields.customfield_10056 if hasattr(issue.fields, 'customfield_10056') else None,
        'Abandoned Reason': issue.fields.customfield_10057 if hasattr(issue.fields, 'customfield_10057') else None
  }
    issue_list.append(issue_dict)

df = pd.DataFrame(issue_list)



df['Updated'] = pd.to_datetime(df['Updated'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Updated'] = df['Updated'].dt.date


df['Created'] = pd.to_datetime(df['Created'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Created'] = df['Created'].dt.date

df['Resolved'] = pd.to_datetime(df['Resolved'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Resolved'] = df['Resolved'].dt.date

df['Start date'] = pd.to_datetime(df['Start date'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Start date'] = df['Start date'].dt.date

df['Dev finished date'] = pd.to_datetime(df['Dev finished date'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Dev finished date'] = df['Dev finished date'].dt.date

df['Went live / completed date'] = pd.to_datetime(df['Went live / completed date'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Went live / completed date'] = df['Went live / completed date'].dt.date

df['Abandoned date'] = pd.to_datetime(df['Abandoned date'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['Abandoned date'] = df['Abandoned date'].dt.date



df.fillna('', inplace=True)



# Export to google sheets

file = 'C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json'
sheet_id = '11HixWcBZ0f53TSIK1LuvzqQtnG7OQy9hBzXu7vSp7GU'
sheet_name = 'Issues'

def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

write_to_gsheet(file, sheet_id, sheet_name, df)
